In [17]:
import cobra as cb
import tempfile
from pathlib import Path
import numpy as np
import statistics

In [2]:
!wget 'http://bigg.ucsd.edu/static/models/iML1515.xml'

--2023-08-10 22:08:33--  http://bigg.ucsd.edu/static/models/iML1515.xml
Resolving bigg.ucsd.edu (bigg.ucsd.edu)... 169.228.33.117
Connecting to bigg.ucsd.edu (bigg.ucsd.edu)|169.228.33.117|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11412293 (11M) [application/xml]
Saving to: ‘iML1515.xml’

iML1515.xml         100%[===================>]  10.88M  4.53MB/s    in 2.4s    

2023-08-10 22:08:35 (4.53 MB/s) - ‘iML1515.xml’ saved [11412293/11412293]



In [3]:
data_path = Path.cwd().resolve()

In [4]:
chassis = cb.io.read_sbml_model(data_path.joinpath('iML1515.xml'))

## Step 1
Generate the metabolites the new pathway will use:

In [5]:
pet_e = cb.Metabolite('pet_e', name='polyethylene terephthalate', compartment='e')
pet_minus_e = cb.Metabolite('petm_e', name='PET(n-1)', compartment='e') # pseudo metabolite for balancing reactions
mhet_e = cb.Metabolite('mhet_e', name='2-Hydroxyethyl terephthalic acid', compartment='e')
tpa_e = cb.Metabolite('tpa_e', name='Terephthalic Acid', compartment='e',
                      formula='C8H6O4')
glycol_e = cb.Metabolite('glycol_e', name='Ethylene Glycol, 1,2-ethanediol', compartment='e',
                         formula='C2H6O2')
glycol_c = cb.Metabolite('glycol_c', name='Ethylene Glycol, 1,2-ethanediol', compartment='c',
                         formula='C2H6O2')
nad_c = chassis.metabolites.get_by_id('nad_c')
nadh_c = chassis.metabolites.get_by_id('nadh_c')
h_c = chassis.metabolites.get_by_id('h_c')


## Step 2
Create the reactions the pathway will use:

In [6]:
# PET depolymerization
petase = cb.Reaction('petase', name='PET Hydrolase', subsystem='PET metabolism',
                     lower_bound= 10.0, upper_bound=1000.0)
petase.add_metabolites({pet_e:-1, chassis.metabolites.get_by_id('h2o_e'):-1,
                        pet_minus_e:1, mhet_e:1})

# Breakdown of PET monomer (MHET) into TPA and ethylene glycol
mhetase = cb.Reaction('mhetase', name='MHET Acylhydrolase', subsystem='PET metabolism',
                      lower_bound=100.0, upper_bound=1000.0)
mhetase.add_metabolites({mhet_e:-1, chassis.metabolites.get_by_id('h2o_e'):-1,
                         tpa_e:1, glycol_e:1})

# Ethylene glycol uptake into cell
gly_diff = cb.Reaction('gly_diff', name='Ethylene Glycol transmembrane diffusion',
                        lower_bound=-1000.0, upper_bound=1000.0)
gly_diff.add_metabolites({glycol_e:-1, glycol_c:1})

# Ethylene glycol oxidization to glycolaldehyde--contact with main metabolic network
gly_oxidase = cb.Reaction('gly_oxidase', name='Ethylene Glycol NAD+ dependent oxidoreductase',
                          lower_bound=-100.0, upper_bound=1000.0)
gly_oxidase.add_metabolites({glycol_c:-1, nad_c:-1, chassis.metabolites.get_by_id('gcald_c'):1,
                             nadh_c:1, h_c:1})

## Step 3
Add the reactions to the main model:

In [7]:
reactions = [petase, mhetase, gly_diff, gly_oxidase]
chassis.add_reactions(reactions)

## Step 4
Add boundary reactions, or unbalanced pseudoreactions which add or remove metabolites from the model

PET needs to be input to the model, the PET minus pseudometabolite needs to be removed, and TPA needs to be removed,
simulating harvesting for industrial use:

In [8]:
chassis.add_boundary(pet_e, type='exchange')

Reaction identifier,EX_pet_e
Name,polyethylene terephthalate exchange
Memory address,0x7f646cade1a0
Stoichiometry,pet_e <=> polyethylene terephthalate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [9]:
chassis.add_boundary(pet_minus_e, type='exchange')

Reaction identifier,EX_petm_e
Name,PET(n-1) exchange
Memory address,0x7f646458a4d0
Stoichiometry,petm_e <=> PET(n-1) <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [10]:
chassis.add_boundary(tpa_e, type='exchange')

Reaction identifier,EX_tpa_e
Name,Terephthalic Acid exchange
Memory address,0x7f64631fd450
Stoichiometry,tpa_e <=> Terephthalic Acid <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Step 5
Calculate optimal flux balance solution:

In [20]:
chassis_solution = chassis.optimize()

In [12]:
chassis_solution

,fluxes,reduced_costs
CYTDK2,0.000000,-1.681434e-02
XPPT,0.000000,2.081668e-17
HXPRT,0.000000,-3.362869e-02
NDPK5,0.353986,7.229549e-17
SHK3Dr,4.993556,-2.303717e-17
...,...,...
gly_diff,714.582189,0.000000e+00
gly_oxidase,714.582189,0.000000e+00
EX_pet_e,-714.582189,0.000000e+00
EX_petm_e,714.582189,0.000000e+00


In [13]:
for reaction in reactions:
    print(f'{reaction.name} flux: {reaction.flux}')

PET Hydrolase flux: 714.5821886141156
MHET Acylhydrolase flux: 714.5821886141156
Ethylene Glycol transmembrane diffusion flux: 714.5821886141155
Ethylene Glycol NAD+ dependent oxidoreductase flux: 714.5821886141155


## Results

We can see here that the PET degradation pathway is highly active during flux balance analysis. This indicates that, at least from a chemical point of view, this metabolism is advantageous to the organism and is likely to be retained. This is expected, given that it not only acts as a carbon source but also reduces NAD+ to NADH, resulting in an energy gain for the cell.

In [14]:
cb.io.write_sbml_model(chassis, str(data_path.joinpath('iML1515_aug.xml')))

In [15]:
cb.io.save_json_model(chassis, str(data_path.joinpath('iML1515_aug.json')))

In [16]:
chassis.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0682,0,0.00%
cl_e,EX_cl_e,0.0682,0,0.00%
co2_e,EX_co2_e,144.8,1,70.71%
cobalt2_e,EX_cobalt2_e,0.0003276,0,0.00%
cu2_e,EX_cu2_e,0.00929,0,0.00%
fe2_e,EX_fe2_e,168.7,0,0.00%
glc__D_e,EX_glc__D_e,10,6,29.29%
h2o_e,EX_h2o_e,429.2,0,0.00%
k_e,EX_k_e,2.557,0,0.00%
mg2_e,EX_mg2_e,0.1137,0,0.00%


## Results
In this example, the model has been instructed to maximize microbial biomass (simulating
the bacteria's primary goal to grow and reproduce). PET metabolism was not included in the
objective function. Despite this, both degradation of PET and secretion of TPA occur at their maximum
possible rates. This suggests that this pathway would be highly active *in vivo*.

Recall that for PET and the PET(n-1) pseudovariable we left the chemical formula blank.
This is why cobra does not report carbon fluxes for these metabolites.

## Test for robustness

In [52]:
def perturb(model):
    medium = model.medium
    
    rng = np.random.default_rng()
    for metabolite, flux in medium.items():
        if metabolite == 'EX_pet_e': continue # keep PET levels in media at maximum
        delta = rng.normal(scale = flux / 3)
        medium[metabolite] = flux - delta
    
    return medium 

In [53]:
test = chassis.copy()
flux_distribution = []
for i in range(10000):
    test.medium = perturb(test)
    optimum = test.optimize()
    flux_distribution.append(optimum.fluxes['petase'])
    

/home/evensong/anaconda3/envs/sim/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/evensong/anaconda3/envs/sim/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/evensong/anaconda3/envs/sim/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/evensong/anaconda3/envs/sim/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/evensong/anaconda3/envs/sim/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/evensong/anaconda3/envs/sim/lib/python3.10/site-packages/cobra/u

ValueError: The lower bound must be less than or equal to the upper bound (1618.1783500308738 <= 1000.0).

In [54]:
mean = statistics.mean(flux_distribution)
stdev = statistics.stdev(flux_distribution)
median = statistics.median(flux_distribution)

print(f'Mean: {mean}\nMedian: {median}\nStandard Deviation: {stdev}')

Mean: 330.09675061830666
Median: 99.99999999999999
Standard Deviation: 294.18203410648687


From these tests, we can see that high activity of the PET degradation pathway is reasonably robust to variations in the environment, maintaining a consistently high flux regardless of perturbations in the environment

In [51]:
test2 = chassis.copy()
medium = test2.medium
medium['EX_glc__D_e'] = 0.0
test2.medium = medium

pet_only = test2.optimize()
test2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0668,0,0.00%
cl_e,EX_cl_e,0.0668,0,0.00%
co2_e,EX_co2_e,144.4,1,100.00%
cobalt2_e,EX_cobalt2_e,0.0003208,0,0.00%
cu2_e,EX_cu2_e,0.009099,0,0.00%
fe2_e,EX_fe2_e,155,0,0.00%
h2o_e,EX_h2o_e,456.2,0,0.00%
k_e,EX_k_e,2.505,0,0.00%
mg2_e,EX_mg2_e,0.1113,0,0.00%
mn2_e,EX_mn2_e,0.008868,0,0.00%


In [55]:
test3 = cb.io.read_sbml_model(data_path.joinpath('iML1515.xml'))

In [56]:
optimum3 = test3.optimize()
test3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.004565,0,0.00%
cl_e,EX_cl_e,0.004565,0,0.00%
cobalt2_e,EX_cobalt2_e,2.192E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006218,0,0.00%
fe2_e,EX_fe2_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1712,0,0.00%
mg2_e,EX_mg2_e,0.007608,0,0.00%
mn2_e,EX_mn2_e,0.000606,0,0.00%
mobd_e,EX_mobd_e,6.139E-06,0,0.00%
